In [11]:
import asyncio
import websockets
import json
from datetime import datetime, timezone, timedelta


def ship_close_to_port(coords):
    global ports
    for port_info in ports.values():
        port = port_info[2:]
        if (coords[0]-port[0])**2 + (coords[1]-port[1])**2 <= 0.2:
            return True
    return False # Ship isn't close to any port


async def connect_ais_stream():
    global ships
    global suspected
    suspected = {}
    ships = []
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {"APIKey": "701e9fda22048c4816c373831750f92821387b06", "BoundingBoxes": [[[-90, -180], [90, 180]]]}
        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        old_ships = {}
        cur_ships = {}
        update_threshold_s = timedelta(seconds=10)
        last_update = datetime.now(timezone.utc)

        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]

            if message_type == "PositionReport":
                ais_message = message["Message"]["PositionReport"]
                #print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Longitude: {ais_message['Longitude']}")
                cur_ships[ais_message["UserID"]] = [ais_message['Longitude'], ais_message['Latitude']]
                
                time_elapsed_s = datetime.now(timezone.utc) - last_update
                if time_elapsed_s >= update_threshold_s:
                    
                    old_ships_keys = set(old_ships.keys())
                    cur_ships_keys = set(cur_ships.keys())
                    missing_ships_keys = old_ships_keys - cur_ships_keys
                    new_ships_keys = cur_ships_keys - old_ships_keys
                    
                    not_in_ports = 0
                    for key in missing_ships_keys:
                        coords = old_ships.get(key, 0)
                        if not ship_close_to_port(coords):
                            not_in_ports += 1
                            ships.append(coords)
                            if key in suspected:
                                suspected[key][1] += 1 # increase count
                            else:
                                suspected[key] = [coords, 1]
                            
                    print(f"[{datetime.now(timezone.utc)}] missing ships: {len(missing_ships_keys)} new ships: {len(new_ships_keys)}")
                    print(f" disapeared: {not_in_ports}, in ports: {len(missing_ships_keys)-not_in_ports}")
                    last_update = datetime.now(timezone.utc)

                    old_ships = cur_ships
                    cur_ships = {}


if __name__ == "__main__":
    await connect_ais_stream()

[2023-03-26 00:57:49.277290+00:00] missing ships: 0 new ships: 2244
 disapeared: 0, in ports: 0
[2023-03-26 00:58:01.280528+00:00] missing ships: 2244 new ships: 2266
 disapeared: 794, in ports: 1450
[2023-03-26 00:58:13.233703+00:00] missing ships: 2266 new ships: 2783
 disapeared: 701, in ports: 1565


CancelledError: 

In [14]:
with open('suspects.json', 'w') as outfile:
    json.dump(suspected, outfile)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

plt.figure(figsize=(20, 14))

map = Basemap(projection='merc', lat_0=0, lon_0=0,
              llcrnrlon=-180, llcrnrlat=-80,
              urcrnrlon=180, urcrnrlat=80)

# Draw coastlines, countries, and states
map.drawcoastlines()
map.drawcountries()
map.drawstates()

ship_lons = []
ship_lats = []

for ship in ships:
    ship_lons.append(ship[0])
    ship_lats.append(ship[1])


x, y = map(ship_lons, ship_lats)
map.plot(x, y, 'ro', markersize=2)

plt.show()

In [ ]:
import mundilib
from PIL import Image

def get_bbox(coords):
    offset = 0.3
    return coords[0] - offset, coords[1] - offset, coords[0] + offset, coords[1] + offset

test = get_bbox(ships[242])

catalog = mundilib.MundiCatalogue()
wms = catalog.get_collection('Sentinel1').mundi_wms('GRD')

bbox = test
projection = 'EPSG:4326'
date = '2020-12-25/2023-03-25'

height = 500
width = 2*height

for layer_name in wms.contents:
    layer = wms[layer_name]
    print(layer.title)
    
    img = wms.getmap(layers = [layer.name],
                     srs = projection,
                     bbox = bbox,
                     size = (width, height),
                     format ='image/png',
                     time = date,
                     showlogo = False)
    
    display(Image.open(img))